## Sentence selection by emotion

In [6]:
'''
Using @EmotionMeter API 
Com autorização do DR.Ricardo.
'''

import requests


def sentimentAnalysis(text,lang='en'):
    body = {

        "messages": text,

        "language": lang

        }

    request = requests.post("http://146.59.159.119:9777", json = body)
    
    return (request.text.encode().decode('unicode-escape') , request.status_code)


In [15]:
fake = "R..P. to the 8 year-old boy who died in Boston's explosions,while running for the Sandy Hook kids. #prayforboston http://t.co/Xmv2E81Lsb"
fact = "❌ No, a missing airliner (Santiago Flight 513) did not mysteriously land 35 years later with a cockpit and passenger cabin full of skeletons."

In [9]:
print(sentimentAnalysis([fake]))

('{"R..P. to the 8 year-old boy who died in Boston\'s explosions,while running for the Sandy Hook kids. #prayforboston http://t.co/Xmv2E81Lsb":{"emotions":{"anger":0.0,"anticipation":0.0,"disgust":0.2,"fear":0.4,"joy":0.0,"sadness":0.2,"surprise":0.2,"trust":0.0},"polarities":{"negative":1.0,"positive":0.0}}}\n', 200)


In [16]:
print(sentimentAnalysis([fact]))

('{"❌ No, a missing airliner (Santiago Flight 513) did not mysteriously land 35 years later with a cockpit and passenger cabin full of skeletons.":{"emotions":{"anger":0.0,"anticipation":1.0,"disgust":0.0,"fear":0.0,"joy":0.0,"sadness":0.0,"surprise":0.0,"trust":0.0},"polarities":{"negative":0.0,"positive":1.0}}}\n', 200)


In [24]:
def emotion_selection(text):
    code = 0
    while (code!=200):
        (sentiments, code) = sentimentAnalysis(text,'en')
    ## Ordenar por ordem decrescente maior emoçao
    
    print(sentiments)
    playload = json.loads(sentiments)

    

    ## Ordenar as frases por aquela que tem maior sadness and surprise
    sortedByEmotion = {k: v for k, v in sorted(playload.items(), key=lambda item: (item[1]['emotions']['sadness'], item[1]['emotions']['surprise']), reverse=True)}
    summary = " "

    i = 0
    for sentence in list(sortedByEmotion.keys()):
        if i ==0:
            summary = sentence
            i = 1
        elif i != 0 and len (summary + sentence) < 280:
            summary = summary + sentence
        else:
            break

    return summary


In [19]:
import json
import pandas as pd
import os

In [32]:
def clean (ListSentences):

    l = [s.replace("\"","\'").replace("\”","\'").replace("\“","\'").replace("\n"," ") for s in ListSentences]

    return l

In [ ]:
data = pd.DataFrame()

## Obtenção dos sentimentos
for file in os.listdir("../../../extractors/Snopes/extractions")[:100]:
    with open("../../../extractors/Snopes/extractions/" + file) as f:
            d = json.load(f)
            f.close()
    text = clean(d['postText'])
    print(d['id'])

    claim = d['evaluation'] + ' , '+ d['allegation']
    emo = emotion_selection(text)    
    dic = {'id': d['id'], 'emotion_sellection': emo, 'claim_sellection': claim }

    data = data.append(dic, ignore_index = True)


data.to_csv('selection_emo_claim.csv')




#### save in the csv file

In [35]:

df_abs = pd.read_csv('summarization.csv', index_col=0)
df_abs.head()

df_ext = pd.read_csv('selection_emo_claim.csv', index_col=0)
df_ext.head()


for ((i_a, r_a),(i_e, r_e))  in zip(df_abs.iterrows(),df_ext.iterrows()):
        if (r_a[0] == r_e[0]): ## tem o mesmo id
                df_abs.loc[i_a,'emotion_selection'] = r_e[1]
                df_abs.loc[i_a,'claim_sellection'] = r_e[2]


df_abs.to_csv('textCore.csv')
